In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv
from datetime import datetime

In [7]:
loc = '/data/SO_predict_DATA'
db_name = '/train_no_dup.db'

In [8]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

In [9]:
con = create_connection(db_name)

In [13]:
if con is not None:
    titile_data = pd.read_sql("""SELECT Title FROM no_dup_train""",con)
else:
    print("Connection Error!!")

In [14]:
titile_data.shape

(4206308, 1)

In [15]:
if con is not None:
    tags_data = pd.read_sql("""SELECT Tags FROM no_dup_train""",con)
else:
    print("Connection Error!!")

In [16]:
tags_data.shape

(4206308, 1)

In [ ]:
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))